In [1]:
from __future__ import division
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
import scipy.optimize as op 

from sklearn.model_selection import train_test_split,StratifiedShuffleSplit 
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn import metrics as metrices

%matplotlib inline

pd.options.display.float_format = '{:.2f}'.format
%config IPCompleter.greedy=True

In [3]:
geisinger_afib_panda = pd.read_csv('geisinger_afib_aflt_patients.csv',low_memory=False)
geisinger_non_afib_panda = pd.read_csv('geisinger_non_afib_aflt_patients.csv',low_memory=False)

In [4]:
geisinger_afib_panda.count()

PT_ID                                                2797
PT_SEX                                               2797
PT_BIRTH_DT                                          2797
PT_DEATH_DT                                          1063
LAST_ACTIVE_DT                                       2797
                                                     ... 
CREATININE_MEDIAN_AFTER_INDEX                        2736
CREATININE_CLOSEST_BEFORE_RECUR_OR_LASTENC           2755
DT_CREATININE_CLOSEST_BEFORE_RECUR_OR_LASTENC        2755
CREATININE_CLOSEST_BEFORE_DEATH_OR_LASTENC           2755
DT_FOR_CREATININE_CLOSEST_BEFORE_DEATH_OR_LASTENC    2755
Length: 284, dtype: int64

In [5]:
#data for 2744 patients's first occurance of afib
data_afib_per_patient_panda = pd.read_csv('df_afib_aflt_no_duplicate_per_patient.csv')

#data for 1062 patients's whose first occurance of afib is before the stroke date
data_afib_diagnosed_before_storke = data_afib_per_patient_panda[data_afib_per_patient_panda['Occurance_in_years'] < 0]
data_afib_diagnosed_before_storke_at_index = data_afib_per_patient_panda[data_afib_per_patient_panda['Occurance_in_years'] == 0]
data_afib_diagnosed_after_storke_within_3_years =  data_afib_per_patient_panda[(data_afib_per_patient_panda['Occurance_in_years'] > 0) & (data_afib_per_patient_panda['Occurance_in_years'] <= 3)]
data_afib_diagnosed_other = data_afib_per_patient_panda[data_afib_per_patient_panda['Occurance_in_years'] > 3]

print(data_afib_diagnosed_before_storke['PT_ID'].nunique())
print(data_afib_diagnosed_before_storke_at_index['PT_ID'].nunique())
print(data_afib_diagnosed_after_storke_within_3_years['PT_ID'].nunique())
print(data_afib_diagnosed_other['PT_ID'].nunique())

1090
841
587
279


# $\color{BLUE}{\text{Patients having afib before stroke index}}$

In [6]:
geisinger_afib_panda_before_stroke = geisinger_afib_panda[geisinger_afib_panda['PT_ID'].isin(data_afib_diagnosed_before_storke['PT_ID'])]


In [7]:
comorbidity_data_afib_before_stroke = geisinger_afib_panda_before_stroke[['PT_ID','PT_SEX','PT_STATUS','FAM_HEART_HIST','FAM_STROKE_HIST','ATRIAL_FIB_PREINDEX','ATRIAL_FLUTTER_PREINDEX','AFIB_FLUTTER_PREINDEX','HYPERTENSION_PREINDEX','MI_PREINDEX','DIABETES_PREINDEX','DYSLIPIDEMIA_PREINDEX','CHF_PREINDEX','HYPERCOAG_STATES_PREINDEX','MILD_LIVER_DIS_PREINDEX','MOD_SEV_LIVER_DIS_PREINDEX','CHRONIC_LIVER_DIS_PREINDEX','CHRONIC_LUNG_DIS_PREINDEX','RHEUM_DIS_PREINDEX','CHRONIC_KIDNEY_DIS_PREINDEX','NEOPLASM_PREINDEX','PERI_VASC_DIS_PREINDEX','PFO_PREINDEX','PAST_ISCHEMIC_STROKE_PREINDEX','PAST_HEMORRHAGIC_STROKE_PREINDEX','ATRIAL_FIB_AT_INDEX','ATRIAL_FLUTTER_AT_INDEX','AFIB_FLUTTER_AT_INDEX','HYPERTENSION_AT_INDEX','MI_AT_INDEX','DIABETES_AT_INDEX','DYSLIPIDEMIA_AT_INDEX','CHF_AT_INDEX','HYPERCOAG_STATES_AT_INDEX','MILD_LIVER_DIS_AT_INDEX','MOD_SEV_LIVER_DIS_AT_INDEX','CHRONIC_LIVER_DIS_AT_INDEX','CHRONIC_LUNG_DIS_AT_INDEX','RHEUM_DIS_AT_INDEX','CHRONIC_KIDNEY_DIS_AT_INDEX','NEOPLASM_AT_INDEX','PERI_VASC_DIS_AT_INDEX','PFO_AT_INDEX','PAST_ISCHEMIC_STROKE_AT_INDEX','PAST_HEMORRHAGIC_STROKE_AT_INDEX']]

In [8]:
comorbidity_data_afib_before_stroke.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1090 entries, 1 to 2796
Data columns (total 45 columns):
PT_ID                               1090 non-null object
PT_SEX                              1090 non-null object
PT_STATUS                           1090 non-null object
FAM_HEART_HIST                      1090 non-null int64
FAM_STROKE_HIST                     1090 non-null int64
ATRIAL_FIB_PREINDEX                 1090 non-null int64
ATRIAL_FLUTTER_PREINDEX             1090 non-null int64
AFIB_FLUTTER_PREINDEX               1090 non-null int64
HYPERTENSION_PREINDEX               1090 non-null int64
MI_PREINDEX                         1090 non-null int64
DIABETES_PREINDEX                   1090 non-null int64
DYSLIPIDEMIA_PREINDEX               1090 non-null int64
CHF_PREINDEX                        1090 non-null int64
HYPERCOAG_STATES_PREINDEX           1090 non-null int64
MILD_LIVER_DIS_PREINDEX             1090 non-null int64
MOD_SEV_LIVER_DIS_PREINDEX          1090 non-null 

### Sex and Family History

In [9]:
SEX =comorbidity_data_afib_before_stroke.groupby(['PT_SEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
SEX['percentage (%)'] = SEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(SEX)
print('=======================================')

FAM_HEART_HIST =comorbidity_data_afib_before_stroke.groupby(['FAM_HEART_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_HEART_HIST['percentage (%)'] = FAM_HEART_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_HEART_HIST)

print('=======================================')
FAM_STROKE_HIST =comorbidity_data_afib_before_stroke.groupby(['FAM_STROKE_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_STROKE_HIST['percentage (%)'] = FAM_STROKE_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_STROKE_HIST)

   PT_SEX  Patients  percentage (%)
0  Female       569           52.20
1    Male       521           47.80
   FAM_HEART_HIST  Patients  percentage (%)
0               0       606           55.60
1               1       484           44.40
   FAM_STROKE_HIST  Patients  percentage (%)
0                0       923           84.68
1                1       167           15.32


### CoMorbidity PREINDEX

In [10]:
ATRIAL_FIB_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['ATRIAL_FIB_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_PREINDEX['percentage (%)'] = ATRIAL_FIB_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_PREINDEX)

print('=======================================')
ATRIAL_FLUTTER_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['ATRIAL_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_PREINDEX['percentage (%)'] = ATRIAL_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_PREINDEX)

print('=======================================')
AFIB_FLUTTER_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['AFIB_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_PREINDEX['percentage (%)'] = AFIB_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_PREINDEX)

print('=======================================')
HYPERTENSION_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['HYPERTENSION_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_PREINDEX['percentage (%)'] = HYPERTENSION_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_PREINDEX)

print('=======================================')
MI_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['MI_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_PREINDEX['percentage (%)'] = MI_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_PREINDEX)

print('=======================================')
DIABETES_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['DIABETES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_PREINDEX['percentage (%)'] = DIABETES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_PREINDEX)

print('=======================================')
DYSLIPIDEMIA_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['DYSLIPIDEMIA_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_PREINDEX['percentage (%)'] = DYSLIPIDEMIA_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_PREINDEX)

print('=======================================')
CHF_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['CHF_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_PREINDEX['percentage (%)'] = CHF_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_PREINDEX)

print('=======================================')
HYPERCOAG_STATES_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['HYPERCOAG_STATES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_PREINDEX['percentage (%)'] = HYPERCOAG_STATES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_PREINDEX)

print('=======================================')
MILD_LIVER_DIS_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['MILD_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_PREINDEX['percentage (%)'] = MILD_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_PREINDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['MOD_SEV_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_PREINDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LIVER_DIS_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['CHRONIC_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_PREINDEX['percentage (%)'] = CHRONIC_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LUNG_DIS_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['CHRONIC_LUNG_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_PREINDEX['percentage (%)'] = CHRONIC_LUNG_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_PREINDEX)

print('=======================================')
RHEUM_DIS_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['RHEUM_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_PREINDEX['percentage (%)'] = RHEUM_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_PREINDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['CHRONIC_KIDNEY_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_PREINDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_PREINDEX)

print('=======================================')
NEOPLASM_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['NEOPLASM_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_PREINDEX['percentage (%)'] = NEOPLASM_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_PREINDEX)

print('=======================================')
PERI_VASC_DIS_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['PERI_VASC_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_PREINDEX['percentage (%)'] = PERI_VASC_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_PREINDEX)

print('=======================================')
PFO_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['PFO_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_PREINDEX['percentage (%)'] = PFO_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_PREINDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['PAST_ISCHEMIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_PREINDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_PREINDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_PREINDEX =comorbidity_data_afib_before_stroke.groupby(['PAST_HEMORRHAGIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_PREINDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_PREINDEX)

   ATRIAL_FIB_PREINDEX  Patients  percentage (%)
0                    0        28            2.57
1                    1      1062           97.43
   ATRIAL_FLUTTER_PREINDEX  Patients  percentage (%)
0                        0       951           87.25
1                        1       139           12.75
   AFIB_FLUTTER_PREINDEX  Patients  percentage (%)
0                      1      1090          100.00
   HYPERTENSION_PREINDEX  Patients  percentage (%)
0                      0       170           15.60
1                      1       920           84.40
   MI_PREINDEX  Patients  percentage (%)
0            0       872           80.00
1            1       218           20.00
   DIABETES_PREINDEX  Patients  percentage (%)
0                  0       689           63.21
1                  1       401           36.79
   DYSLIPIDEMIA_PREINDEX  Patients  percentage (%)
0                      0       333           30.55
1                      1       757           69.45
   CHF_PREINDEX  Patie

### CoMorbidity AT INDEX

In [11]:
ATRIAL_FIB_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['ATRIAL_FIB_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_AT_INDEX['percentage (%)'] = ATRIAL_FIB_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_AT_INDEX)

print('=======================================')
ATRIAL_FLUTTER_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['ATRIAL_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_AT_INDEX['percentage (%)'] = ATRIAL_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_AT_INDEX)

print('=======================================')
AFIB_FLUTTER_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['AFIB_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_AT_INDEX['percentage (%)'] = AFIB_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_AT_INDEX)

print('=======================================')
HYPERTENSION_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['HYPERTENSION_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_AT_INDEX['percentage (%)'] = HYPERTENSION_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_AT_INDEX)

print('=======================================')
MI_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['MI_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_AT_INDEX['percentage (%)'] = MI_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_AT_INDEX)

print('=======================================')
DIABETES_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['DIABETES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_AT_INDEX['percentage (%)'] = DIABETES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_AT_INDEX)

print('=======================================')
DYSLIPIDEMIA_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['DYSLIPIDEMIA_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_AT_INDEX['percentage (%)'] = DYSLIPIDEMIA_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_AT_INDEX)

print('=======================================')
CHF_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['CHF_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_AT_INDEX['percentage (%)'] = CHF_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_AT_INDEX)

print('=======================================')
HYPERCOAG_STATES_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['HYPERCOAG_STATES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_AT_INDEX['percentage (%)'] = HYPERCOAG_STATES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_AT_INDEX)

print('=======================================')
MILD_LIVER_DIS_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['MILD_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_AT_INDEX['percentage (%)'] = MILD_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_AT_INDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['MOD_SEV_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_AT_INDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LIVER_DIS_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['CHRONIC_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LUNG_DIS_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['CHRONIC_LUNG_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LUNG_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_AT_INDEX)

print('=======================================')
RHEUM_DIS_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['RHEUM_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_AT_INDEX['percentage (%)'] = RHEUM_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_AT_INDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['CHRONIC_KIDNEY_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_AT_INDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_AT_INDEX)

print('=======================================')
NEOPLASM_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['NEOPLASM_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_AT_INDEX['percentage (%)'] = NEOPLASM_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_AT_INDEX)

print('=======================================')
PERI_VASC_DIS_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['PERI_VASC_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_AT_INDEX['percentage (%)'] = PERI_VASC_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_AT_INDEX)

print('=======================================')
PFO_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['PFO_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_AT_INDEX['percentage (%)'] = PFO_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_AT_INDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['PAST_ISCHEMIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_AT_INDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_AT_INDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_AT_INDEX =comorbidity_data_afib_before_stroke.groupby(['PAST_HEMORRHAGIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_AT_INDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_AT_INDEX)

   ATRIAL_FIB_AT_INDEX  Patients  percentage (%)
0                    0        19            1.74
1                    1      1071           98.26
   ATRIAL_FLUTTER_AT_INDEX  Patients  percentage (%)
0                        0       940           86.24
1                        1       150           13.76
   AFIB_FLUTTER_AT_INDEX  Patients  percentage (%)
0                      1      1090          100.00
   HYPERTENSION_AT_INDEX  Patients  percentage (%)
0                      0       108            9.91
1                      1       982           90.09
   MI_AT_INDEX  Patients  percentage (%)
0            0       850           77.98
1            1       240           22.02
   DIABETES_AT_INDEX  Patients  percentage (%)
0                  0       665           61.01
1                  1       425           38.99
   DYSLIPIDEMIA_AT_INDEX  Patients  percentage (%)
0                      0       273           25.05
1                      1       817           74.95
   CHF_AT_INDEX  Patie

# $\color{BLUE}{\text{Patients having afib at stroke index}}$

In [12]:
geisinger_afib_panda_at_stroke = geisinger_afib_panda[geisinger_afib_panda['PT_ID'].isin(data_afib_diagnosed_before_storke_at_index['PT_ID'])]
comorbidity_data_afib_at_stroke = geisinger_afib_panda_at_stroke[['PT_ID','PT_SEX','PT_STATUS','FAM_HEART_HIST','FAM_STROKE_HIST','ATRIAL_FIB_PREINDEX','ATRIAL_FLUTTER_PREINDEX','AFIB_FLUTTER_PREINDEX','HYPERTENSION_PREINDEX','MI_PREINDEX','DIABETES_PREINDEX','DYSLIPIDEMIA_PREINDEX','CHF_PREINDEX','HYPERCOAG_STATES_PREINDEX','MILD_LIVER_DIS_PREINDEX','MOD_SEV_LIVER_DIS_PREINDEX','CHRONIC_LIVER_DIS_PREINDEX','CHRONIC_LUNG_DIS_PREINDEX','RHEUM_DIS_PREINDEX','CHRONIC_KIDNEY_DIS_PREINDEX','NEOPLASM_PREINDEX','PERI_VASC_DIS_PREINDEX','PFO_PREINDEX','PAST_ISCHEMIC_STROKE_PREINDEX','PAST_HEMORRHAGIC_STROKE_PREINDEX','ATRIAL_FIB_AT_INDEX','ATRIAL_FLUTTER_AT_INDEX','AFIB_FLUTTER_AT_INDEX','HYPERTENSION_AT_INDEX','MI_AT_INDEX','DIABETES_AT_INDEX','DYSLIPIDEMIA_AT_INDEX','CHF_AT_INDEX','HYPERCOAG_STATES_AT_INDEX','MILD_LIVER_DIS_AT_INDEX','MOD_SEV_LIVER_DIS_AT_INDEX','CHRONIC_LIVER_DIS_AT_INDEX','CHRONIC_LUNG_DIS_AT_INDEX','RHEUM_DIS_AT_INDEX','CHRONIC_KIDNEY_DIS_AT_INDEX','NEOPLASM_AT_INDEX','PERI_VASC_DIS_AT_INDEX','PFO_AT_INDEX','PAST_ISCHEMIC_STROKE_AT_INDEX','PAST_HEMORRHAGIC_STROKE_AT_INDEX']]
comorbidity_data_afib_at_stroke.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 841 entries, 0 to 2795
Data columns (total 45 columns):
PT_ID                               841 non-null object
PT_SEX                              841 non-null object
PT_STATUS                           841 non-null object
FAM_HEART_HIST                      841 non-null int64
FAM_STROKE_HIST                     841 non-null int64
ATRIAL_FIB_PREINDEX                 841 non-null int64
ATRIAL_FLUTTER_PREINDEX             841 non-null int64
AFIB_FLUTTER_PREINDEX               841 non-null int64
HYPERTENSION_PREINDEX               841 non-null int64
MI_PREINDEX                         841 non-null int64
DIABETES_PREINDEX                   841 non-null int64
DYSLIPIDEMIA_PREINDEX               841 non-null int64
CHF_PREINDEX                        841 non-null int64
HYPERCOAG_STATES_PREINDEX           841 non-null int64
MILD_LIVER_DIS_PREINDEX             841 non-null int64
MOD_SEV_LIVER_DIS_PREINDEX          841 non-null int64
CHRONIC_LIV

### Sex and Family History

In [13]:
SEX =comorbidity_data_afib_at_stroke.groupby(['PT_SEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
SEX['percentage (%)'] = SEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(SEX)
print('=======================================')

FAM_HEART_HIST =comorbidity_data_afib_at_stroke.groupby(['FAM_HEART_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_HEART_HIST['percentage (%)'] = FAM_HEART_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_HEART_HIST)

print('=======================================')
FAM_STROKE_HIST =comorbidity_data_afib_at_stroke.groupby(['FAM_STROKE_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_STROKE_HIST['percentage (%)'] = FAM_STROKE_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_STROKE_HIST)

   PT_SEX  Patients  percentage (%)
0  Female       444           52.79
1    Male       397           47.21
   FAM_HEART_HIST  Patients  percentage (%)
0               0       595           70.75
1               1       246           29.25
   FAM_STROKE_HIST  Patients  percentage (%)
0                0       740           87.99
1                1       101           12.01


### CoMorbidity PREINDEX

In [14]:
ATRIAL_FIB_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['ATRIAL_FIB_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_PREINDEX['percentage (%)'] = ATRIAL_FIB_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_PREINDEX)

print('=======================================')
ATRIAL_FLUTTER_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['ATRIAL_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_PREINDEX['percentage (%)'] = ATRIAL_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_PREINDEX)

print('=======================================')
AFIB_FLUTTER_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['AFIB_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_PREINDEX['percentage (%)'] = AFIB_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_PREINDEX)

print('=======================================')
HYPERTENSION_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['HYPERTENSION_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_PREINDEX['percentage (%)'] = HYPERTENSION_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_PREINDEX)

print('=======================================')
MI_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['MI_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_PREINDEX['percentage (%)'] = MI_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_PREINDEX)

print('=======================================')
DIABETES_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['DIABETES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_PREINDEX['percentage (%)'] = DIABETES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_PREINDEX)

print('=======================================')
DYSLIPIDEMIA_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['DYSLIPIDEMIA_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_PREINDEX['percentage (%)'] = DYSLIPIDEMIA_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_PREINDEX)

print('=======================================')
CHF_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['CHF_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_PREINDEX['percentage (%)'] = CHF_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_PREINDEX)

print('=======================================')
HYPERCOAG_STATES_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['HYPERCOAG_STATES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_PREINDEX['percentage (%)'] = HYPERCOAG_STATES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_PREINDEX)

print('=======================================')
MILD_LIVER_DIS_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['MILD_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_PREINDEX['percentage (%)'] = MILD_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_PREINDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['MOD_SEV_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_PREINDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LIVER_DIS_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['CHRONIC_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_PREINDEX['percentage (%)'] = CHRONIC_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LUNG_DIS_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['CHRONIC_LUNG_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_PREINDEX['percentage (%)'] = CHRONIC_LUNG_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_PREINDEX)

print('=======================================')
RHEUM_DIS_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['RHEUM_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_PREINDEX['percentage (%)'] = RHEUM_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_PREINDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['CHRONIC_KIDNEY_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_PREINDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_PREINDEX)

print('=======================================')
NEOPLASM_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['NEOPLASM_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_PREINDEX['percentage (%)'] = NEOPLASM_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_PREINDEX)

print('=======================================')
PERI_VASC_DIS_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['PERI_VASC_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_PREINDEX['percentage (%)'] = PERI_VASC_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_PREINDEX)

print('=======================================')
PFO_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['PFO_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_PREINDEX['percentage (%)'] = PFO_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_PREINDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['PAST_ISCHEMIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_PREINDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_PREINDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_PREINDEX =comorbidity_data_afib_at_stroke.groupby(['PAST_HEMORRHAGIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_PREINDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_PREINDEX)

   ATRIAL_FIB_PREINDEX  Patients  percentage (%)
0                    0       841          100.00
   ATRIAL_FLUTTER_PREINDEX  Patients  percentage (%)
0                        0       841          100.00
   AFIB_FLUTTER_PREINDEX  Patients  percentage (%)
0                      0       841          100.00
   HYPERTENSION_PREINDEX  Patients  percentage (%)
0                      0       505           60.05
1                      1       336           39.95
   MI_PREINDEX  Patients  percentage (%)
0            0       794           94.41
1            1        47            5.59
   DIABETES_PREINDEX  Patients  percentage (%)
0                  0       702           83.47
1                  1       139           16.53
   DYSLIPIDEMIA_PREINDEX  Patients  percentage (%)
0                      0       559           66.47
1                      1       282           33.53
   CHF_PREINDEX  Patients  percentage (%)
0             0       806           95.84
1             1        35            4.1

### CoMorbidity AT INDEX

In [15]:
ATRIAL_FIB_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['ATRIAL_FIB_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_AT_INDEX['percentage (%)'] = ATRIAL_FIB_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_AT_INDEX)

print('=======================================')
ATRIAL_FLUTTER_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['ATRIAL_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_AT_INDEX['percentage (%)'] = ATRIAL_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_AT_INDEX)

print('=======================================')
AFIB_FLUTTER_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['AFIB_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_AT_INDEX['percentage (%)'] = AFIB_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_AT_INDEX)

print('=======================================')
HYPERTENSION_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['HYPERTENSION_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_AT_INDEX['percentage (%)'] = HYPERTENSION_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_AT_INDEX)

print('=======================================')
MI_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['MI_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_AT_INDEX['percentage (%)'] = MI_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_AT_INDEX)

print('=======================================')
DIABETES_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['DIABETES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_AT_INDEX['percentage (%)'] = DIABETES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_AT_INDEX)

print('=======================================')
DYSLIPIDEMIA_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['DYSLIPIDEMIA_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_AT_INDEX['percentage (%)'] = DYSLIPIDEMIA_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_AT_INDEX)

print('=======================================')
CHF_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['CHF_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_AT_INDEX['percentage (%)'] = CHF_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_AT_INDEX)

print('=======================================')
HYPERCOAG_STATES_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['HYPERCOAG_STATES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_AT_INDEX['percentage (%)'] = HYPERCOAG_STATES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_AT_INDEX)

print('=======================================')
MILD_LIVER_DIS_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['MILD_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_AT_INDEX['percentage (%)'] = MILD_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_AT_INDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['MOD_SEV_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_AT_INDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LIVER_DIS_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['CHRONIC_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LUNG_DIS_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['CHRONIC_LUNG_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LUNG_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_AT_INDEX)

print('=======================================')
RHEUM_DIS_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['RHEUM_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_AT_INDEX['percentage (%)'] = RHEUM_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_AT_INDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['CHRONIC_KIDNEY_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_AT_INDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_AT_INDEX)

print('=======================================')
NEOPLASM_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['NEOPLASM_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_AT_INDEX['percentage (%)'] = NEOPLASM_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_AT_INDEX)

print('=======================================')
PERI_VASC_DIS_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['PERI_VASC_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_AT_INDEX['percentage (%)'] = PERI_VASC_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_AT_INDEX)

print('=======================================')
PFO_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['PFO_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_AT_INDEX['percentage (%)'] = PFO_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_AT_INDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['PAST_ISCHEMIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_AT_INDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_AT_INDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_AT_INDEX =comorbidity_data_afib_at_stroke.groupby(['PAST_HEMORRHAGIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_AT_INDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_AT_INDEX)

   ATRIAL_FIB_AT_INDEX  Patients  percentage (%)
0                    0        32            3.80
1                    1       809           96.20
   ATRIAL_FLUTTER_AT_INDEX  Patients  percentage (%)
0                        0       756           89.89
1                        1        85           10.11
   AFIB_FLUTTER_AT_INDEX  Patients  percentage (%)
0                      1       841          100.00
   HYPERTENSION_AT_INDEX  Patients  percentage (%)
0                      0       136           16.17
1                      1       705           83.83
   MI_AT_INDEX  Patients  percentage (%)
0            0       755           89.77
1            1        86           10.23
   DIABETES_AT_INDEX  Patients  percentage (%)
0                  0       584           69.44
1                  1       257           30.56
   DYSLIPIDEMIA_AT_INDEX  Patients  percentage (%)
0                      0       342           40.67
1                      1       499           59.33
   CHF_AT_INDEX  Patie

# $\color{BLUE}{\text{Patients having afib within 3 years after stroke index}}$

In [16]:
geisinger_afib_panda_after_storke_within_3_years = geisinger_afib_panda[geisinger_afib_panda['PT_ID'].isin(data_afib_diagnosed_after_storke_within_3_years['PT_ID'])]
comorbidity_data_afib_after_storke_within_3_years = geisinger_afib_panda_after_storke_within_3_years[['PT_ID','PT_SEX','PT_STATUS','FAM_HEART_HIST','FAM_STROKE_HIST','ATRIAL_FIB_PREINDEX','ATRIAL_FLUTTER_PREINDEX','AFIB_FLUTTER_PREINDEX','HYPERTENSION_PREINDEX','MI_PREINDEX','DIABETES_PREINDEX','DYSLIPIDEMIA_PREINDEX','CHF_PREINDEX','HYPERCOAG_STATES_PREINDEX','MILD_LIVER_DIS_PREINDEX','MOD_SEV_LIVER_DIS_PREINDEX','CHRONIC_LIVER_DIS_PREINDEX','CHRONIC_LUNG_DIS_PREINDEX','RHEUM_DIS_PREINDEX','CHRONIC_KIDNEY_DIS_PREINDEX','NEOPLASM_PREINDEX','PERI_VASC_DIS_PREINDEX','PFO_PREINDEX','PAST_ISCHEMIC_STROKE_PREINDEX','PAST_HEMORRHAGIC_STROKE_PREINDEX','ATRIAL_FIB_AT_INDEX','ATRIAL_FLUTTER_AT_INDEX','AFIB_FLUTTER_AT_INDEX','HYPERTENSION_AT_INDEX','MI_AT_INDEX','DIABETES_AT_INDEX','DYSLIPIDEMIA_AT_INDEX','CHF_AT_INDEX','HYPERCOAG_STATES_AT_INDEX','MILD_LIVER_DIS_AT_INDEX','MOD_SEV_LIVER_DIS_AT_INDEX','CHRONIC_LIVER_DIS_AT_INDEX','CHRONIC_LUNG_DIS_AT_INDEX','RHEUM_DIS_AT_INDEX','CHRONIC_KIDNEY_DIS_AT_INDEX','NEOPLASM_AT_INDEX','PERI_VASC_DIS_AT_INDEX','PFO_AT_INDEX','PAST_ISCHEMIC_STROKE_AT_INDEX','PAST_HEMORRHAGIC_STROKE_AT_INDEX']]
comorbidity_data_afib_after_storke_within_3_years.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 587 entries, 4 to 2789
Data columns (total 45 columns):
PT_ID                               587 non-null object
PT_SEX                              587 non-null object
PT_STATUS                           587 non-null object
FAM_HEART_HIST                      587 non-null int64
FAM_STROKE_HIST                     587 non-null int64
ATRIAL_FIB_PREINDEX                 587 non-null int64
ATRIAL_FLUTTER_PREINDEX             587 non-null int64
AFIB_FLUTTER_PREINDEX               587 non-null int64
HYPERTENSION_PREINDEX               587 non-null int64
MI_PREINDEX                         587 non-null int64
DIABETES_PREINDEX                   587 non-null int64
DYSLIPIDEMIA_PREINDEX               587 non-null int64
CHF_PREINDEX                        587 non-null int64
HYPERCOAG_STATES_PREINDEX           587 non-null int64
MILD_LIVER_DIS_PREINDEX             587 non-null int64
MOD_SEV_LIVER_DIS_PREINDEX          587 non-null int64
CHRONIC_LIV

### Sex and Family History

In [17]:
SEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['PT_SEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
SEX['percentage (%)'] = SEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(SEX)
print('=======================================')

FAM_HEART_HIST =comorbidity_data_afib_after_storke_within_3_years.groupby(['FAM_HEART_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_HEART_HIST['percentage (%)'] = FAM_HEART_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_HEART_HIST)

print('=======================================')
FAM_STROKE_HIST =comorbidity_data_afib_after_storke_within_3_years.groupby(['FAM_STROKE_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_STROKE_HIST['percentage (%)'] = FAM_STROKE_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_STROKE_HIST)

   PT_SEX  Patients  percentage (%)
0  Female       309           52.64
1    Male       278           47.36
   FAM_HEART_HIST  Patients  percentage (%)
0               0       366           62.35
1               1       221           37.65
   FAM_STROKE_HIST  Patients  percentage (%)
0                0       506           86.20
1                1        81           13.80


### CoMorbidity PREINDEX

In [18]:
ATRIAL_FIB_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['ATRIAL_FIB_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_PREINDEX['percentage (%)'] = ATRIAL_FIB_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_PREINDEX)

print('=======================================')
ATRIAL_FLUTTER_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['ATRIAL_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_PREINDEX['percentage (%)'] = ATRIAL_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_PREINDEX)

print('=======================================')
AFIB_FLUTTER_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['AFIB_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_PREINDEX['percentage (%)'] = AFIB_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_PREINDEX)

print('=======================================')
HYPERTENSION_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['HYPERTENSION_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_PREINDEX['percentage (%)'] = HYPERTENSION_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_PREINDEX)

print('=======================================')
MI_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['MI_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_PREINDEX['percentage (%)'] = MI_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_PREINDEX)

print('=======================================')
DIABETES_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['DIABETES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_PREINDEX['percentage (%)'] = DIABETES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_PREINDEX)

print('=======================================')
DYSLIPIDEMIA_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['DYSLIPIDEMIA_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_PREINDEX['percentage (%)'] = DYSLIPIDEMIA_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_PREINDEX)

print('=======================================')
CHF_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['CHF_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_PREINDEX['percentage (%)'] = CHF_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_PREINDEX)

print('=======================================')
HYPERCOAG_STATES_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['HYPERCOAG_STATES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_PREINDEX['percentage (%)'] = HYPERCOAG_STATES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_PREINDEX)

print('=======================================')
MILD_LIVER_DIS_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['MILD_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_PREINDEX['percentage (%)'] = MILD_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_PREINDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['MOD_SEV_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_PREINDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LIVER_DIS_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['CHRONIC_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_PREINDEX['percentage (%)'] = CHRONIC_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LUNG_DIS_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['CHRONIC_LUNG_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_PREINDEX['percentage (%)'] = CHRONIC_LUNG_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_PREINDEX)

print('=======================================')
RHEUM_DIS_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['RHEUM_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_PREINDEX['percentage (%)'] = RHEUM_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_PREINDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['CHRONIC_KIDNEY_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_PREINDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_PREINDEX)

print('=======================================')
NEOPLASM_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['NEOPLASM_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_PREINDEX['percentage (%)'] = NEOPLASM_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_PREINDEX)

print('=======================================')
PERI_VASC_DIS_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['PERI_VASC_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_PREINDEX['percentage (%)'] = PERI_VASC_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_PREINDEX)

print('=======================================')
PFO_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['PFO_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_PREINDEX['percentage (%)'] = PFO_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_PREINDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['PAST_ISCHEMIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_PREINDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_PREINDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_PREINDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['PAST_HEMORRHAGIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_PREINDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_PREINDEX)

   ATRIAL_FIB_PREINDEX  Patients  percentage (%)
0                    0       587          100.00
   ATRIAL_FLUTTER_PREINDEX  Patients  percentage (%)
0                        0       587          100.00
   AFIB_FLUTTER_PREINDEX  Patients  percentage (%)
0                      0       587          100.00
   HYPERTENSION_PREINDEX  Patients  percentage (%)
0                      0       273           46.51
1                      1       314           53.49
   MI_PREINDEX  Patients  percentage (%)
0            0       536           91.31
1            1        51            8.69
   DIABETES_PREINDEX  Patients  percentage (%)
0                  0       424           72.23
1                  1       163           27.77
   DYSLIPIDEMIA_PREINDEX  Patients  percentage (%)
0                      0       323           55.03
1                      1       264           44.97
   CHF_PREINDEX  Patients  percentage (%)
0             0       524           89.27
1             1        63           10.7

### CoMorbidity AT INDEX

In [19]:
ATRIAL_FIB_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['ATRIAL_FIB_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_AT_INDEX['percentage (%)'] = ATRIAL_FIB_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_AT_INDEX)

print('=======================================')
ATRIAL_FLUTTER_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['ATRIAL_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_AT_INDEX['percentage (%)'] = ATRIAL_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_AT_INDEX)

print('=======================================')
AFIB_FLUTTER_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['AFIB_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_AT_INDEX['percentage (%)'] = AFIB_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_AT_INDEX)

print('=======================================')
HYPERTENSION_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['HYPERTENSION_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_AT_INDEX['percentage (%)'] = HYPERTENSION_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_AT_INDEX)

print('=======================================')
MI_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['MI_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_AT_INDEX['percentage (%)'] = MI_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_AT_INDEX)

print('=======================================')
DIABETES_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['DIABETES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_AT_INDEX['percentage (%)'] = DIABETES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_AT_INDEX)

print('=======================================')
DYSLIPIDEMIA_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['DYSLIPIDEMIA_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_AT_INDEX['percentage (%)'] = DYSLIPIDEMIA_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_AT_INDEX)

print('=======================================')
CHF_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['CHF_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_AT_INDEX['percentage (%)'] = CHF_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_AT_INDEX)

print('=======================================')
HYPERCOAG_STATES_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['HYPERCOAG_STATES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_AT_INDEX['percentage (%)'] = HYPERCOAG_STATES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_AT_INDEX)

print('=======================================')
MILD_LIVER_DIS_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['MILD_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_AT_INDEX['percentage (%)'] = MILD_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_AT_INDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['MOD_SEV_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_AT_INDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LIVER_DIS_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['CHRONIC_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LUNG_DIS_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['CHRONIC_LUNG_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LUNG_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_AT_INDEX)

print('=======================================')
RHEUM_DIS_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['RHEUM_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_AT_INDEX['percentage (%)'] = RHEUM_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_AT_INDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['CHRONIC_KIDNEY_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_AT_INDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_AT_INDEX)

print('=======================================')
NEOPLASM_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['NEOPLASM_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_AT_INDEX['percentage (%)'] = NEOPLASM_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_AT_INDEX)

print('=======================================')
PERI_VASC_DIS_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['PERI_VASC_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_AT_INDEX['percentage (%)'] = PERI_VASC_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_AT_INDEX)

print('=======================================')
PFO_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['PFO_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_AT_INDEX['percentage (%)'] = PFO_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_AT_INDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['PAST_ISCHEMIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_AT_INDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_AT_INDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_AT_INDEX =comorbidity_data_afib_after_storke_within_3_years.groupby(['PAST_HEMORRHAGIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_AT_INDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_AT_INDEX)

   ATRIAL_FIB_AT_INDEX  Patients  percentage (%)
0                    0       587          100.00
   ATRIAL_FLUTTER_AT_INDEX  Patients  percentage (%)
0                        0       587          100.00
   AFIB_FLUTTER_AT_INDEX  Patients  percentage (%)
0                      0       587          100.00
   HYPERTENSION_AT_INDEX  Patients  percentage (%)
0                      0       155           26.41
1                      1       432           73.59
   MI_AT_INDEX  Patients  percentage (%)
0            0       517           88.07
1            1        70           11.93
   DIABETES_AT_INDEX  Patients  percentage (%)
0                  0       385           65.59
1                  1       202           34.41
   DYSLIPIDEMIA_AT_INDEX  Patients  percentage (%)
0                      0       229           39.01
1                      1       358           60.99
   CHF_AT_INDEX  Patients  percentage (%)
0             0       492           83.82
1             1        95           16.1

# $\color{BLUE}{\text{Patients having afib 3 years after stroke index}}$

In [20]:
geisinger_afib_panda_other = geisinger_afib_panda[geisinger_afib_panda['PT_ID'].isin(data_afib_diagnosed_other['PT_ID'])]
comorbidity_data_afib_other = geisinger_afib_panda_other[['PT_ID','PT_SEX','PT_STATUS','FAM_HEART_HIST','FAM_STROKE_HIST','ATRIAL_FIB_PREINDEX','ATRIAL_FLUTTER_PREINDEX','AFIB_FLUTTER_PREINDEX','HYPERTENSION_PREINDEX','MI_PREINDEX','DIABETES_PREINDEX','DYSLIPIDEMIA_PREINDEX','CHF_PREINDEX','HYPERCOAG_STATES_PREINDEX','MILD_LIVER_DIS_PREINDEX','MOD_SEV_LIVER_DIS_PREINDEX','CHRONIC_LIVER_DIS_PREINDEX','CHRONIC_LUNG_DIS_PREINDEX','RHEUM_DIS_PREINDEX','CHRONIC_KIDNEY_DIS_PREINDEX','NEOPLASM_PREINDEX','PERI_VASC_DIS_PREINDEX','PFO_PREINDEX','PAST_ISCHEMIC_STROKE_PREINDEX','PAST_HEMORRHAGIC_STROKE_PREINDEX','ATRIAL_FIB_AT_INDEX','ATRIAL_FLUTTER_AT_INDEX','AFIB_FLUTTER_AT_INDEX','HYPERTENSION_AT_INDEX','MI_AT_INDEX','DIABETES_AT_INDEX','DYSLIPIDEMIA_AT_INDEX','CHF_AT_INDEX','HYPERCOAG_STATES_AT_INDEX','MILD_LIVER_DIS_AT_INDEX','MOD_SEV_LIVER_DIS_AT_INDEX','CHRONIC_LIVER_DIS_AT_INDEX','CHRONIC_LUNG_DIS_AT_INDEX','RHEUM_DIS_AT_INDEX','CHRONIC_KIDNEY_DIS_AT_INDEX','NEOPLASM_AT_INDEX','PERI_VASC_DIS_AT_INDEX','PFO_AT_INDEX','PAST_ISCHEMIC_STROKE_AT_INDEX','PAST_HEMORRHAGIC_STROKE_AT_INDEX']]
comorbidity_data_afib_other.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 279 entries, 2 to 2793
Data columns (total 45 columns):
PT_ID                               279 non-null object
PT_SEX                              279 non-null object
PT_STATUS                           279 non-null object
FAM_HEART_HIST                      279 non-null int64
FAM_STROKE_HIST                     279 non-null int64
ATRIAL_FIB_PREINDEX                 279 non-null int64
ATRIAL_FLUTTER_PREINDEX             279 non-null int64
AFIB_FLUTTER_PREINDEX               279 non-null int64
HYPERTENSION_PREINDEX               279 non-null int64
MI_PREINDEX                         279 non-null int64
DIABETES_PREINDEX                   279 non-null int64
DYSLIPIDEMIA_PREINDEX               279 non-null int64
CHF_PREINDEX                        279 non-null int64
HYPERCOAG_STATES_PREINDEX           279 non-null int64
MILD_LIVER_DIS_PREINDEX             279 non-null int64
MOD_SEV_LIVER_DIS_PREINDEX          279 non-null int64
CHRONIC_LIV

### Sex and Family History

In [21]:
SEX =comorbidity_data_afib_other.groupby(['PT_SEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
SEX['percentage (%)'] = SEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(SEX)

print('=======================================')
FAM_HEART_HIST =comorbidity_data_afib_other.groupby(['FAM_HEART_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_HEART_HIST['percentage (%)'] = FAM_HEART_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_HEART_HIST)

print('=======================================')
FAM_STROKE_HIST =comorbidity_data_afib_other.groupby(['FAM_STROKE_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_STROKE_HIST['percentage (%)'] = FAM_STROKE_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_STROKE_HIST)

   PT_SEX  Patients  percentage (%)
0  Female       146           52.33
1    Male       133           47.67
   FAM_HEART_HIST  Patients  percentage (%)
0               0       158           56.63
1               1       121           43.37
   FAM_STROKE_HIST  Patients  percentage (%)
0                0       225           80.65
1                1        54           19.35


### CoMorbidity PREINDEX

In [22]:
ATRIAL_FIB_PREINDEX =comorbidity_data_afib_other.groupby(['ATRIAL_FIB_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_PREINDEX['percentage (%)'] = ATRIAL_FIB_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_PREINDEX)

print('=======================================')
ATRIAL_FLUTTER_PREINDEX =comorbidity_data_afib_other.groupby(['ATRIAL_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_PREINDEX['percentage (%)'] = ATRIAL_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_PREINDEX)

print('=======================================')
AFIB_FLUTTER_PREINDEX =comorbidity_data_afib_other.groupby(['AFIB_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_PREINDEX['percentage (%)'] = AFIB_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_PREINDEX)

print('=======================================')
HYPERTENSION_PREINDEX =comorbidity_data_afib_other.groupby(['HYPERTENSION_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_PREINDEX['percentage (%)'] = HYPERTENSION_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_PREINDEX)

print('=======================================')
MI_PREINDEX =comorbidity_data_afib_other.groupby(['MI_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_PREINDEX['percentage (%)'] = MI_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_PREINDEX)

print('=======================================')
DIABETES_PREINDEX =comorbidity_data_afib_other.groupby(['DIABETES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_PREINDEX['percentage (%)'] = DIABETES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_PREINDEX)

print('=======================================')
DYSLIPIDEMIA_PREINDEX =comorbidity_data_afib_other.groupby(['DYSLIPIDEMIA_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_PREINDEX['percentage (%)'] = DYSLIPIDEMIA_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_PREINDEX)

print('=======================================')
CHF_PREINDEX =comorbidity_data_afib_other.groupby(['CHF_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_PREINDEX['percentage (%)'] = CHF_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_PREINDEX)

print('=======================================')
HYPERCOAG_STATES_PREINDEX =comorbidity_data_afib_other.groupby(['HYPERCOAG_STATES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_PREINDEX['percentage (%)'] = HYPERCOAG_STATES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_PREINDEX)

print('=======================================')
MILD_LIVER_DIS_PREINDEX =comorbidity_data_afib_other.groupby(['MILD_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_PREINDEX['percentage (%)'] = MILD_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_PREINDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_PREINDEX =comorbidity_data_afib_other.groupby(['MOD_SEV_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_PREINDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LIVER_DIS_PREINDEX =comorbidity_data_afib_other.groupby(['CHRONIC_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_PREINDEX['percentage (%)'] = CHRONIC_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LUNG_DIS_PREINDEX =comorbidity_data_afib_other.groupby(['CHRONIC_LUNG_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_PREINDEX['percentage (%)'] = CHRONIC_LUNG_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_PREINDEX)

print('=======================================')
RHEUM_DIS_PREINDEX =comorbidity_data_afib_other.groupby(['RHEUM_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_PREINDEX['percentage (%)'] = RHEUM_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_PREINDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_PREINDEX =comorbidity_data_afib_other.groupby(['CHRONIC_KIDNEY_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_PREINDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_PREINDEX)

print('=======================================')
NEOPLASM_PREINDEX =comorbidity_data_afib_other.groupby(['NEOPLASM_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_PREINDEX['percentage (%)'] = NEOPLASM_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_PREINDEX)

print('=======================================')
PERI_VASC_DIS_PREINDEX =comorbidity_data_afib_other.groupby(['PERI_VASC_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_PREINDEX['percentage (%)'] = PERI_VASC_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_PREINDEX)

print('=======================================')
PFO_PREINDEX =comorbidity_data_afib_other.groupby(['PFO_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_PREINDEX['percentage (%)'] = PFO_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_PREINDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_PREINDEX =comorbidity_data_afib_other.groupby(['PAST_ISCHEMIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_PREINDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_PREINDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_PREINDEX =comorbidity_data_afib_other.groupby(['PAST_HEMORRHAGIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_PREINDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_PREINDEX)

   ATRIAL_FIB_PREINDEX  Patients  percentage (%)
0                    0       279          100.00
   ATRIAL_FLUTTER_PREINDEX  Patients  percentage (%)
0                        0       279          100.00
   AFIB_FLUTTER_PREINDEX  Patients  percentage (%)
0                      0       279          100.00
   HYPERTENSION_PREINDEX  Patients  percentage (%)
0                      0       139           49.82
1                      1       140           50.18
   MI_PREINDEX  Patients  percentage (%)
0            0       267           95.70
1            1        12            4.30
   DIABETES_PREINDEX  Patients  percentage (%)
0                  0       205           73.48
1                  1        74           26.52
   DYSLIPIDEMIA_PREINDEX  Patients  percentage (%)
0                      0       160           57.35
1                      1       119           42.65
   CHF_PREINDEX  Patients  percentage (%)
0             0       260           93.19
1             1        19            6.8

### CoMorbidity AT INDEX

In [23]:
ATRIAL_FIB_AT_INDEX =comorbidity_data_afib_other.groupby(['ATRIAL_FIB_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_AT_INDEX['percentage (%)'] = ATRIAL_FIB_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_AT_INDEX)

print('=======================================')
ATRIAL_FLUTTER_AT_INDEX =comorbidity_data_afib_other.groupby(['ATRIAL_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_AT_INDEX['percentage (%)'] = ATRIAL_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_AT_INDEX)

print('=======================================')
AFIB_FLUTTER_AT_INDEX =comorbidity_data_afib_other.groupby(['AFIB_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_AT_INDEX['percentage (%)'] = AFIB_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_AT_INDEX)

print('=======================================')
HYPERTENSION_AT_INDEX =comorbidity_data_afib_other.groupby(['HYPERTENSION_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_AT_INDEX['percentage (%)'] = HYPERTENSION_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_AT_INDEX)

print('=======================================')
MI_AT_INDEX =comorbidity_data_afib_other.groupby(['MI_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_AT_INDEX['percentage (%)'] = MI_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_AT_INDEX)

print('=======================================')
DIABETES_AT_INDEX =comorbidity_data_afib_other.groupby(['DIABETES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_AT_INDEX['percentage (%)'] = DIABETES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_AT_INDEX)

print('=======================================')
DYSLIPIDEMIA_AT_INDEX =comorbidity_data_afib_other.groupby(['DYSLIPIDEMIA_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_AT_INDEX['percentage (%)'] = DYSLIPIDEMIA_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_AT_INDEX)

print('=======================================')
CHF_AT_INDEX =comorbidity_data_afib_other.groupby(['CHF_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_AT_INDEX['percentage (%)'] = CHF_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_AT_INDEX)

print('=======================================')
HYPERCOAG_STATES_AT_INDEX =comorbidity_data_afib_other.groupby(['HYPERCOAG_STATES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_AT_INDEX['percentage (%)'] = HYPERCOAG_STATES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_AT_INDEX)

print('=======================================')
MILD_LIVER_DIS_AT_INDEX =comorbidity_data_afib_other.groupby(['MILD_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_AT_INDEX['percentage (%)'] = MILD_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_AT_INDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_AT_INDEX =comorbidity_data_afib_other.groupby(['MOD_SEV_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_AT_INDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LIVER_DIS_AT_INDEX =comorbidity_data_afib_other.groupby(['CHRONIC_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LUNG_DIS_AT_INDEX =comorbidity_data_afib_other.groupby(['CHRONIC_LUNG_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LUNG_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_AT_INDEX)

print('=======================================')
RHEUM_DIS_AT_INDEX =comorbidity_data_afib_other.groupby(['RHEUM_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_AT_INDEX['percentage (%)'] = RHEUM_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_AT_INDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_AT_INDEX =comorbidity_data_afib_other.groupby(['CHRONIC_KIDNEY_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_AT_INDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_AT_INDEX)

print('=======================================')
NEOPLASM_AT_INDEX =comorbidity_data_afib_other.groupby(['NEOPLASM_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_AT_INDEX['percentage (%)'] = NEOPLASM_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_AT_INDEX)

print('=======================================')
PERI_VASC_DIS_AT_INDEX =comorbidity_data_afib_other.groupby(['PERI_VASC_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_AT_INDEX['percentage (%)'] = PERI_VASC_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_AT_INDEX)

print('=======================================')
PFO_AT_INDEX =comorbidity_data_afib_other.groupby(['PFO_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_AT_INDEX['percentage (%)'] = PFO_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_AT_INDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_AT_INDEX =comorbidity_data_afib_other.groupby(['PAST_ISCHEMIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_AT_INDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_AT_INDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_AT_INDEX =comorbidity_data_afib_other.groupby(['PAST_HEMORRHAGIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_AT_INDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_AT_INDEX)

   ATRIAL_FIB_AT_INDEX  Patients  percentage (%)
0                    0       279          100.00
   ATRIAL_FLUTTER_AT_INDEX  Patients  percentage (%)
0                        0       279          100.00
   AFIB_FLUTTER_AT_INDEX  Patients  percentage (%)
0                      0       279          100.00
   HYPERTENSION_AT_INDEX  Patients  percentage (%)
0                      0        80           28.67
1                      1       199           71.33
   MI_AT_INDEX  Patients  percentage (%)
0            0       258           92.47
1            1        21            7.53
   DIABETES_AT_INDEX  Patients  percentage (%)
0                  0       181           64.87
1                  1        98           35.13
   DYSLIPIDEMIA_AT_INDEX  Patients  percentage (%)
0                      0       102           36.56
1                      1       177           63.44
   CHF_AT_INDEX  Patients  percentage (%)
0             0       257           92.11
1             1        22            7.8

# $\color{RED}{\text{Patients having no Afib at all}}$

In [24]:
comorbidity_data_non_afib = geisinger_non_afib_panda[['PT_ID','PT_SEX','PT_STATUS','FAM_HEART_HIST','FAM_STROKE_HIST','ATRIAL_FIB_PREINDEX','ATRIAL_FLUTTER_PREINDEX','AFIB_FLUTTER_PREINDEX','HYPERTENSION_PREINDEX','MI_PREINDEX','DIABETES_PREINDEX','DYSLIPIDEMIA_PREINDEX','CHF_PREINDEX','HYPERCOAG_STATES_PREINDEX','MILD_LIVER_DIS_PREINDEX','MOD_SEV_LIVER_DIS_PREINDEX','CHRONIC_LIVER_DIS_PREINDEX','CHRONIC_LUNG_DIS_PREINDEX','RHEUM_DIS_PREINDEX','CHRONIC_KIDNEY_DIS_PREINDEX','NEOPLASM_PREINDEX','PERI_VASC_DIS_PREINDEX','PFO_PREINDEX','PAST_ISCHEMIC_STROKE_PREINDEX','PAST_HEMORRHAGIC_STROKE_PREINDEX','ATRIAL_FIB_AT_INDEX','ATRIAL_FLUTTER_AT_INDEX','AFIB_FLUTTER_AT_INDEX','HYPERTENSION_AT_INDEX','MI_AT_INDEX','DIABETES_AT_INDEX','DYSLIPIDEMIA_AT_INDEX','CHF_AT_INDEX','HYPERCOAG_STATES_AT_INDEX','MILD_LIVER_DIS_AT_INDEX','MOD_SEV_LIVER_DIS_AT_INDEX','CHRONIC_LIVER_DIS_AT_INDEX','CHRONIC_LUNG_DIS_AT_INDEX','RHEUM_DIS_AT_INDEX','CHRONIC_KIDNEY_DIS_AT_INDEX','NEOPLASM_AT_INDEX','PERI_VASC_DIS_AT_INDEX','PFO_AT_INDEX','PAST_ISCHEMIC_STROKE_AT_INDEX','PAST_HEMORRHAGIC_STROKE_AT_INDEX']]
comorbidity_data_non_afib.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6132 entries, 0 to 6131
Data columns (total 45 columns):
PT_ID                               6132 non-null object
PT_SEX                              6132 non-null object
PT_STATUS                           6132 non-null object
FAM_HEART_HIST                      6132 non-null int64
FAM_STROKE_HIST                     6132 non-null int64
ATRIAL_FIB_PREINDEX                 6132 non-null int64
ATRIAL_FLUTTER_PREINDEX             6132 non-null int64
AFIB_FLUTTER_PREINDEX               6132 non-null int64
HYPERTENSION_PREINDEX               6132 non-null int64
MI_PREINDEX                         6132 non-null int64
DIABETES_PREINDEX                   6132 non-null int64
DYSLIPIDEMIA_PREINDEX               6132 non-null int64
CHF_PREINDEX                        6132 non-null int64
HYPERCOAG_STATES_PREINDEX           6132 non-null int64
MILD_LIVER_DIS_PREINDEX             6132 non-null int64
MOD_SEV_LIVER_DIS_PREINDEX          6132 non-null 

### Sex and Family History

In [25]:
SEX =comorbidity_data_non_afib.groupby(['PT_SEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
SEX['percentage (%)'] = SEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(SEX)

print('=======================================')
FAM_HEART_HIST =comorbidity_data_non_afib.groupby(['FAM_HEART_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_HEART_HIST['percentage (%)'] = FAM_HEART_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_HEART_HIST)

print('=======================================')
FAM_STROKE_HIST =comorbidity_data_non_afib.groupby(['FAM_STROKE_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_STROKE_HIST['percentage (%)'] = FAM_STROKE_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_STROKE_HIST)

   PT_SEX  Patients  percentage (%)
0  Female      2851           46.49
1    Male      3281           53.51
   FAM_HEART_HIST  Patients  percentage (%)
0               0      4036           65.82
1               1      2096           34.18
   FAM_STROKE_HIST  Patients  percentage (%)
0                0      5331           86.94
1                1       801           13.06


### CoMorbidity PREINDEX

In [26]:
ATRIAL_FIB_PREINDEX =comorbidity_data_non_afib.groupby(['ATRIAL_FIB_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_PREINDEX['percentage (%)'] = ATRIAL_FIB_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_PREINDEX)

print('=======================================')
ATRIAL_FLUTTER_PREINDEX =comorbidity_data_non_afib.groupby(['ATRIAL_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_PREINDEX['percentage (%)'] = ATRIAL_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_PREINDEX)

print('=======================================')
AFIB_FLUTTER_PREINDEX =comorbidity_data_non_afib.groupby(['AFIB_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_PREINDEX['percentage (%)'] = AFIB_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_PREINDEX)

print('=======================================')
HYPERTENSION_PREINDEX =comorbidity_data_non_afib.groupby(['HYPERTENSION_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_PREINDEX['percentage (%)'] = HYPERTENSION_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_PREINDEX)

print('=======================================')
MI_PREINDEX =comorbidity_data_non_afib.groupby(['MI_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_PREINDEX['percentage (%)'] = MI_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_PREINDEX)

print('=======================================')
DIABETES_PREINDEX =comorbidity_data_non_afib.groupby(['DIABETES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_PREINDEX['percentage (%)'] = DIABETES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_PREINDEX)

print('=======================================')
DYSLIPIDEMIA_PREINDEX =comorbidity_data_non_afib.groupby(['DYSLIPIDEMIA_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_PREINDEX['percentage (%)'] = DYSLIPIDEMIA_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_PREINDEX)

print('=======================================')
CHF_PREINDEX =comorbidity_data_non_afib.groupby(['CHF_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_PREINDEX['percentage (%)'] = CHF_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_PREINDEX)

print('=======================================')
HYPERCOAG_STATES_PREINDEX =comorbidity_data_non_afib.groupby(['HYPERCOAG_STATES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_PREINDEX['percentage (%)'] = HYPERCOAG_STATES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_PREINDEX)

print('=======================================')
MILD_LIVER_DIS_PREINDEX =comorbidity_data_non_afib.groupby(['MILD_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_PREINDEX['percentage (%)'] = MILD_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_PREINDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_PREINDEX =comorbidity_data_non_afib.groupby(['MOD_SEV_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_PREINDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LIVER_DIS_PREINDEX =comorbidity_data_non_afib.groupby(['CHRONIC_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_PREINDEX['percentage (%)'] = CHRONIC_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LUNG_DIS_PREINDEX =comorbidity_data_non_afib.groupby(['CHRONIC_LUNG_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_PREINDEX['percentage (%)'] = CHRONIC_LUNG_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_PREINDEX)

print('=======================================')
RHEUM_DIS_PREINDEX =comorbidity_data_non_afib.groupby(['RHEUM_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_PREINDEX['percentage (%)'] = RHEUM_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_PREINDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_PREINDEX =comorbidity_data_non_afib.groupby(['CHRONIC_KIDNEY_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_PREINDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_PREINDEX)

print('=======================================')
NEOPLASM_PREINDEX =comorbidity_data_non_afib.groupby(['NEOPLASM_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_PREINDEX['percentage (%)'] = NEOPLASM_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_PREINDEX)

print('=======================================')
PERI_VASC_DIS_PREINDEX =comorbidity_data_non_afib.groupby(['PERI_VASC_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_PREINDEX['percentage (%)'] = PERI_VASC_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_PREINDEX)

print('=======================================')
PFO_PREINDEX =comorbidity_data_non_afib.groupby(['PFO_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_PREINDEX['percentage (%)'] = PFO_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_PREINDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_PREINDEX =comorbidity_data_non_afib.groupby(['PAST_ISCHEMIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_PREINDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_PREINDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_PREINDEX =comorbidity_data_non_afib.groupby(['PAST_HEMORRHAGIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_PREINDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_PREINDEX)

   ATRIAL_FIB_PREINDEX  Patients  percentage (%)
0                    0      6132          100.00
   ATRIAL_FLUTTER_PREINDEX  Patients  percentage (%)
0                        0      6132          100.00
   AFIB_FLUTTER_PREINDEX  Patients  percentage (%)
0                      0      6132          100.00
   HYPERTENSION_PREINDEX  Patients  percentage (%)
0                      0      3403           55.50
1                      1      2729           44.50
   MI_PREINDEX  Patients  percentage (%)
0            0      5708           93.09
1            1       424            6.91
   DIABETES_PREINDEX  Patients  percentage (%)
0                  0      4794           78.18
1                  1      1338           21.82
   DYSLIPIDEMIA_PREINDEX  Patients  percentage (%)
0                      0      3746           61.09
1                      1      2386           38.91
   CHF_PREINDEX  Patients  percentage (%)
0             0      5796           94.52
1             1       336            5.4

### CoMorbidity AT INDEX

In [27]:
ATRIAL_FIB_AT_INDEX =comorbidity_data_non_afib.groupby(['ATRIAL_FIB_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_AT_INDEX['percentage (%)'] = ATRIAL_FIB_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_AT_INDEX)

print('=======================================')
ATRIAL_FLUTTER_AT_INDEX =comorbidity_data_non_afib.groupby(['ATRIAL_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_AT_INDEX['percentage (%)'] = ATRIAL_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_AT_INDEX)

print('=======================================')
AFIB_FLUTTER_AT_INDEX =comorbidity_data_non_afib.groupby(['AFIB_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_AT_INDEX['percentage (%)'] = AFIB_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_AT_INDEX)

print('=======================================')
HYPERTENSION_AT_INDEX =comorbidity_data_non_afib.groupby(['HYPERTENSION_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_AT_INDEX['percentage (%)'] = HYPERTENSION_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_AT_INDEX)

print('=======================================')
MI_AT_INDEX =comorbidity_data_non_afib.groupby(['MI_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_AT_INDEX['percentage (%)'] = MI_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_AT_INDEX)

print('=======================================')
DIABETES_AT_INDEX =comorbidity_data_non_afib.groupby(['DIABETES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_AT_INDEX['percentage (%)'] = DIABETES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_AT_INDEX)

print('=======================================')
DYSLIPIDEMIA_AT_INDEX =comorbidity_data_non_afib.groupby(['DYSLIPIDEMIA_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_AT_INDEX['percentage (%)'] = DYSLIPIDEMIA_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_AT_INDEX)

print('=======================================')
CHF_AT_INDEX =comorbidity_data_non_afib.groupby(['CHF_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_AT_INDEX['percentage (%)'] = CHF_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_AT_INDEX)

print('=======================================')
HYPERCOAG_STATES_AT_INDEX =comorbidity_data_non_afib.groupby(['HYPERCOAG_STATES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_AT_INDEX['percentage (%)'] = HYPERCOAG_STATES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_AT_INDEX)

print('=======================================')
MILD_LIVER_DIS_AT_INDEX =comorbidity_data_non_afib.groupby(['MILD_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_AT_INDEX['percentage (%)'] = MILD_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_AT_INDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_AT_INDEX =comorbidity_data_non_afib.groupby(['MOD_SEV_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_AT_INDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LIVER_DIS_AT_INDEX =comorbidity_data_non_afib.groupby(['CHRONIC_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LUNG_DIS_AT_INDEX =comorbidity_data_non_afib.groupby(['CHRONIC_LUNG_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LUNG_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_AT_INDEX)

print('=======================================')
RHEUM_DIS_AT_INDEX =comorbidity_data_non_afib.groupby(['RHEUM_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_AT_INDEX['percentage (%)'] = RHEUM_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_AT_INDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_AT_INDEX =comorbidity_data_non_afib.groupby(['CHRONIC_KIDNEY_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_AT_INDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_AT_INDEX)

print('=======================================')
NEOPLASM_AT_INDEX =comorbidity_data_non_afib.groupby(['NEOPLASM_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_AT_INDEX['percentage (%)'] = NEOPLASM_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_AT_INDEX)

print('=======================================')
PERI_VASC_DIS_AT_INDEX =comorbidity_data_non_afib.groupby(['PERI_VASC_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_AT_INDEX['percentage (%)'] = PERI_VASC_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_AT_INDEX)

print('=======================================')
PFO_AT_INDEX =comorbidity_data_non_afib.groupby(['PFO_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_AT_INDEX['percentage (%)'] = PFO_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_AT_INDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_AT_INDEX =comorbidity_data_non_afib.groupby(['PAST_ISCHEMIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_AT_INDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_AT_INDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_AT_INDEX =comorbidity_data_non_afib.groupby(['PAST_HEMORRHAGIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_AT_INDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_AT_INDEX)

   ATRIAL_FIB_AT_INDEX  Patients  percentage (%)
0                    0      6131           99.98
1                    1         1            0.02
   ATRIAL_FLUTTER_AT_INDEX  Patients  percentage (%)
0                        0      6132          100.00
   AFIB_FLUTTER_AT_INDEX  Patients  percentage (%)
0                      0      6131           99.98
1                      1         1            0.02
   HYPERTENSION_AT_INDEX  Patients  percentage (%)
0                      0      1738           28.34
1                      1      4394           71.66
   MI_AT_INDEX  Patients  percentage (%)
0            0      5518           89.99
1            1       614           10.01
   DIABETES_AT_INDEX  Patients  percentage (%)
0                  0      4216           68.75
1                  1      1916           31.25
   DYSLIPIDEMIA_AT_INDEX  Patients  percentage (%)
0                      0      2424           39.53
1                      1      3708           60.47
   CHF_AT_INDEX  Patient

# having at least one follow up within 3 years and no afib

In [32]:
data_full_non_afib_patients = pd.read_csv('data_full_non_afib_aflt_patients.csv')

In [33]:
print(data_full_non_afib_patients['PT_ID'].nunique())
data_full_non_afib_patients['INDEX_DT'] = pd.to_datetime(data_full_non_afib_patients['INDEX_DT'])
data_full_non_afib_patients['ENC_DT'] = pd.to_datetime(data_full_non_afib_patients['ENC_DT'])
data_full_non_afib_patients['Difference'] = (data_full_non_afib_patients['ENC_DT'] - data_full_non_afib_patients['INDEX_DT'])
data_full_non_afib_patients['Difference_in_years'] = (data_full_non_afib_patients['ENC_DT'] - data_full_non_afib_patients['INDEX_DT'])/np.timedelta64(1,'Y')
data_full_non_afib_encounter_after_stroke = data_full_non_afib_patients[data_full_non_afib_patients['Difference_in_years'] > 0]
print(data_full_non_afib_encounter_after_stroke['PT_ID'].nunique())

6132
5739


In [34]:
data_full_non_afib_patients_no_afib_3_years_follow_up = data_full_non_afib_encounter_after_stroke[data_full_non_afib_encounter_after_stroke['Difference_in_years'] > 3]
print(data_full_non_afib_patients_no_afib_3_years_follow_up['PT_ID'].nunique())
geisinger_non_afib_panda_3_year_followup = geisinger_non_afib_panda[geisinger_non_afib_panda['PT_ID'].isin(data_full_non_afib_patients_no_afib_3_years_follow_up['PT_ID'].unique())]
geisinger_non_afib_panda_3_year_followup['PT_ID'].nunique()

2176


2176

In [35]:
comorbidity_data_non_afib_3_year_followup = geisinger_non_afib_panda_3_year_followup[['PT_ID','PT_SEX','PT_STATUS','FAM_HEART_HIST','FAM_STROKE_HIST','ATRIAL_FIB_PREINDEX','ATRIAL_FLUTTER_PREINDEX','AFIB_FLUTTER_PREINDEX','HYPERTENSION_PREINDEX','MI_PREINDEX','DIABETES_PREINDEX','DYSLIPIDEMIA_PREINDEX','CHF_PREINDEX','HYPERCOAG_STATES_PREINDEX','MILD_LIVER_DIS_PREINDEX','MOD_SEV_LIVER_DIS_PREINDEX','CHRONIC_LIVER_DIS_PREINDEX','CHRONIC_LUNG_DIS_PREINDEX','RHEUM_DIS_PREINDEX','CHRONIC_KIDNEY_DIS_PREINDEX','NEOPLASM_PREINDEX','PERI_VASC_DIS_PREINDEX','PFO_PREINDEX','PAST_ISCHEMIC_STROKE_PREINDEX','PAST_HEMORRHAGIC_STROKE_PREINDEX','ATRIAL_FIB_AT_INDEX','ATRIAL_FLUTTER_AT_INDEX','AFIB_FLUTTER_AT_INDEX','HYPERTENSION_AT_INDEX','MI_AT_INDEX','DIABETES_AT_INDEX','DYSLIPIDEMIA_AT_INDEX','CHF_AT_INDEX','HYPERCOAG_STATES_AT_INDEX','MILD_LIVER_DIS_AT_INDEX','MOD_SEV_LIVER_DIS_AT_INDEX','CHRONIC_LIVER_DIS_AT_INDEX','CHRONIC_LUNG_DIS_AT_INDEX','RHEUM_DIS_AT_INDEX','CHRONIC_KIDNEY_DIS_AT_INDEX','NEOPLASM_AT_INDEX','PERI_VASC_DIS_AT_INDEX','PFO_AT_INDEX','PAST_ISCHEMIC_STROKE_AT_INDEX','PAST_HEMORRHAGIC_STROKE_AT_INDEX']]
comorbidity_data_non_afib_3_year_followup['PT_ID'].nunique()

2176

### Sex and Family History

In [36]:
SEX =comorbidity_data_non_afib_3_year_followup.groupby(['PT_SEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
SEX['percentage (%)'] = SEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(SEX)

print('=======================================')
FAM_HEART_HIST =comorbidity_data_non_afib_3_year_followup.groupby(['FAM_HEART_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_HEART_HIST['percentage (%)'] = FAM_HEART_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_HEART_HIST)

print('=======================================')
FAM_STROKE_HIST =comorbidity_data_non_afib_3_year_followup.groupby(['FAM_STROKE_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_STROKE_HIST['percentage (%)'] = FAM_STROKE_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_STROKE_HIST)

   PT_SEX  Patients  percentage (%)
0  Female      1003           46.09
1    Male      1173           53.91
   FAM_HEART_HIST  Patients  percentage (%)
0               0      1222           56.16
1               1       954           43.84
   FAM_STROKE_HIST  Patients  percentage (%)
0                0      1826           83.92
1                1       350           16.08


### CoMorbidity PREINDEX

In [37]:
ATRIAL_FIB_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['ATRIAL_FIB_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_PREINDEX['percentage (%)'] = ATRIAL_FIB_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_PREINDEX)

print('=======================================')
ATRIAL_FLUTTER_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['ATRIAL_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_PREINDEX['percentage (%)'] = ATRIAL_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_PREINDEX)

print('=======================================')
AFIB_FLUTTER_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['AFIB_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_PREINDEX['percentage (%)'] = AFIB_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_PREINDEX)

print('=======================================')
HYPERTENSION_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['HYPERTENSION_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_PREINDEX['percentage (%)'] = HYPERTENSION_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_PREINDEX)

print('=======================================')
MI_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['MI_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_PREINDEX['percentage (%)'] = MI_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_PREINDEX)

print('=======================================')
DIABETES_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['DIABETES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_PREINDEX['percentage (%)'] = DIABETES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_PREINDEX)

print('=======================================')
DYSLIPIDEMIA_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['DYSLIPIDEMIA_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_PREINDEX['percentage (%)'] = DYSLIPIDEMIA_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_PREINDEX)

print('=======================================')
CHF_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['CHF_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_PREINDEX['percentage (%)'] = CHF_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_PREINDEX)

print('=======================================')
HYPERCOAG_STATES_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['HYPERCOAG_STATES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_PREINDEX['percentage (%)'] = HYPERCOAG_STATES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_PREINDEX)

print('=======================================')
MILD_LIVER_DIS_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['MILD_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_PREINDEX['percentage (%)'] = MILD_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_PREINDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['MOD_SEV_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_PREINDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LIVER_DIS_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['CHRONIC_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_PREINDEX['percentage (%)'] = CHRONIC_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LUNG_DIS_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['CHRONIC_LUNG_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_PREINDEX['percentage (%)'] = CHRONIC_LUNG_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_PREINDEX)

print('=======================================')
RHEUM_DIS_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['RHEUM_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_PREINDEX['percentage (%)'] = RHEUM_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_PREINDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['CHRONIC_KIDNEY_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_PREINDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_PREINDEX)

print('=======================================')
NEOPLASM_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['NEOPLASM_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_PREINDEX['percentage (%)'] = NEOPLASM_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_PREINDEX)

print('=======================================')
PERI_VASC_DIS_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['PERI_VASC_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_PREINDEX['percentage (%)'] = PERI_VASC_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_PREINDEX)

print('=======================================')
PFO_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['PFO_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_PREINDEX['percentage (%)'] = PFO_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_PREINDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['PAST_ISCHEMIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_PREINDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_PREINDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_PREINDEX =comorbidity_data_non_afib_3_year_followup.groupby(['PAST_HEMORRHAGIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_PREINDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_PREINDEX)

   ATRIAL_FIB_PREINDEX  Patients  percentage (%)
0                    0      2176          100.00
   ATRIAL_FLUTTER_PREINDEX  Patients  percentage (%)
0                        0      2176          100.00
   AFIB_FLUTTER_PREINDEX  Patients  percentage (%)
0                      0      2176          100.00
   HYPERTENSION_PREINDEX  Patients  percentage (%)
0                      0      1194           54.87
1                      1       982           45.13
   MI_PREINDEX  Patients  percentage (%)
0            0      2049           94.16
1            1       127            5.84
   DIABETES_PREINDEX  Patients  percentage (%)
0                  0      1727           79.37
1                  1       449           20.63
   DYSLIPIDEMIA_PREINDEX  Patients  percentage (%)
0                      0      1299           59.70
1                      1       877           40.30
   CHF_PREINDEX  Patients  percentage (%)
0             0      2102           96.60
1             1        74            3.4

### CoMorbidity AT INDEX

In [38]:
ATRIAL_FIB_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['ATRIAL_FIB_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_AT_INDEX['percentage (%)'] = ATRIAL_FIB_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_AT_INDEX)

print('=======================================')
ATRIAL_FLUTTER_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['ATRIAL_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_AT_INDEX['percentage (%)'] = ATRIAL_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_AT_INDEX)

print('=======================================')
AFIB_FLUTTER_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['AFIB_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_AT_INDEX['percentage (%)'] = AFIB_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_AT_INDEX)

print('=======================================')
HYPERTENSION_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['HYPERTENSION_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_AT_INDEX['percentage (%)'] = HYPERTENSION_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_AT_INDEX)

print('=======================================')
MI_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['MI_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_AT_INDEX['percentage (%)'] = MI_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_AT_INDEX)

print('=======================================')
DIABETES_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['DIABETES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_AT_INDEX['percentage (%)'] = DIABETES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_AT_INDEX)

print('=======================================')
DYSLIPIDEMIA_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['DYSLIPIDEMIA_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_AT_INDEX['percentage (%)'] = DYSLIPIDEMIA_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_AT_INDEX)

print('=======================================')
CHF_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['CHF_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_AT_INDEX['percentage (%)'] = CHF_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_AT_INDEX)

print('=======================================')
HYPERCOAG_STATES_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['HYPERCOAG_STATES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_AT_INDEX['percentage (%)'] = HYPERCOAG_STATES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_AT_INDEX)

print('=======================================')
MILD_LIVER_DIS_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['MILD_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_AT_INDEX['percentage (%)'] = MILD_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_AT_INDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['MOD_SEV_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_AT_INDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LIVER_DIS_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['CHRONIC_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LUNG_DIS_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['CHRONIC_LUNG_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LUNG_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_AT_INDEX)

print('=======================================')
RHEUM_DIS_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['RHEUM_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_AT_INDEX['percentage (%)'] = RHEUM_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_AT_INDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['CHRONIC_KIDNEY_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_AT_INDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_AT_INDEX)

print('=======================================')
NEOPLASM_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['NEOPLASM_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_AT_INDEX['percentage (%)'] = NEOPLASM_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_AT_INDEX)

print('=======================================')
PERI_VASC_DIS_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['PERI_VASC_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_AT_INDEX['percentage (%)'] = PERI_VASC_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_AT_INDEX)

print('=======================================')
PFO_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['PFO_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_AT_INDEX['percentage (%)'] = PFO_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_AT_INDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['PAST_ISCHEMIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_AT_INDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_AT_INDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_AT_INDEX =comorbidity_data_non_afib_3_year_followup.groupby(['PAST_HEMORRHAGIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_AT_INDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_AT_INDEX)

   ATRIAL_FIB_AT_INDEX  Patients  percentage (%)
0                    0      2176          100.00
   ATRIAL_FLUTTER_AT_INDEX  Patients  percentage (%)
0                        0      2176          100.00
   AFIB_FLUTTER_AT_INDEX  Patients  percentage (%)
0                      0      2176          100.00
   HYPERTENSION_AT_INDEX  Patients  percentage (%)
0                      0       633           29.09
1                      1      1543           70.91
   MI_AT_INDEX  Patients  percentage (%)
0            0      1988           91.36
1            1       188            8.64
   DIABETES_AT_INDEX  Patients  percentage (%)
0                  0      1534           70.50
1                  1       642           29.50
   DYSLIPIDEMIA_AT_INDEX  Patients  percentage (%)
0                      0       819           37.64
1                      1      1357           62.36
   CHF_AT_INDEX  Patients  percentage (%)
0             0      2071           95.17
1             1       105            4.8

# having at least one follow up within 5 years and no afib

In [39]:
data_full_non_afib_patients_no_afib_5_years_follow_up = data_full_non_afib_encounter_after_stroke[data_full_non_afib_encounter_after_stroke['Difference_in_years'] > 5]
print(data_full_non_afib_patients_no_afib_5_years_follow_up['PT_ID'].nunique())
geisinger_non_afib_panda_5_year_followup = geisinger_non_afib_panda[geisinger_non_afib_panda['PT_ID'].isin(data_full_non_afib_patients_no_afib_5_years_follow_up['PT_ID'].unique())]
geisinger_non_afib_panda_5_year_followup['PT_ID'].nunique()

1302


1302

In [40]:
comorbidity_data_non_afib_5_year_followup = geisinger_non_afib_panda_5_year_followup[['PT_ID','PT_SEX','PT_STATUS','FAM_HEART_HIST','FAM_STROKE_HIST','ATRIAL_FIB_PREINDEX','ATRIAL_FLUTTER_PREINDEX','AFIB_FLUTTER_PREINDEX','HYPERTENSION_PREINDEX','MI_PREINDEX','DIABETES_PREINDEX','DYSLIPIDEMIA_PREINDEX','CHF_PREINDEX','HYPERCOAG_STATES_PREINDEX','MILD_LIVER_DIS_PREINDEX','MOD_SEV_LIVER_DIS_PREINDEX','CHRONIC_LIVER_DIS_PREINDEX','CHRONIC_LUNG_DIS_PREINDEX','RHEUM_DIS_PREINDEX','CHRONIC_KIDNEY_DIS_PREINDEX','NEOPLASM_PREINDEX','PERI_VASC_DIS_PREINDEX','PFO_PREINDEX','PAST_ISCHEMIC_STROKE_PREINDEX','PAST_HEMORRHAGIC_STROKE_PREINDEX','ATRIAL_FIB_AT_INDEX','ATRIAL_FLUTTER_AT_INDEX','AFIB_FLUTTER_AT_INDEX','HYPERTENSION_AT_INDEX','MI_AT_INDEX','DIABETES_AT_INDEX','DYSLIPIDEMIA_AT_INDEX','CHF_AT_INDEX','HYPERCOAG_STATES_AT_INDEX','MILD_LIVER_DIS_AT_INDEX','MOD_SEV_LIVER_DIS_AT_INDEX','CHRONIC_LIVER_DIS_AT_INDEX','CHRONIC_LUNG_DIS_AT_INDEX','RHEUM_DIS_AT_INDEX','CHRONIC_KIDNEY_DIS_AT_INDEX','NEOPLASM_AT_INDEX','PERI_VASC_DIS_AT_INDEX','PFO_AT_INDEX','PAST_ISCHEMIC_STROKE_AT_INDEX','PAST_HEMORRHAGIC_STROKE_AT_INDEX']]
comorbidity_data_non_afib_5_year_followup['PT_ID'].nunique()

1302

### Sex and Family History

In [41]:
SEX =comorbidity_data_non_afib_5_year_followup.groupby(['PT_SEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
SEX['percentage (%)'] = SEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(SEX)

print('=======================================')
FAM_HEART_HIST =comorbidity_data_non_afib_5_year_followup.groupby(['FAM_HEART_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_HEART_HIST['percentage (%)'] = FAM_HEART_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_HEART_HIST)

print('=======================================')
FAM_STROKE_HIST =comorbidity_data_non_afib_5_year_followup.groupby(['FAM_STROKE_HIST']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
FAM_STROKE_HIST['percentage (%)'] = FAM_STROKE_HIST[['Patients']].apply(lambda x: x *100/ x.sum())
print(FAM_STROKE_HIST)

   PT_SEX  Patients  percentage (%)
0  Female       613           47.08
1    Male       689           52.92
   FAM_HEART_HIST  Patients  percentage (%)
0               0       709           54.45
1               1       593           45.55
   FAM_STROKE_HIST  Patients  percentage (%)
0                0      1099           84.41
1                1       203           15.59


### CoMorbidity PREINDEX

In [42]:
ATRIAL_FIB_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['ATRIAL_FIB_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_PREINDEX['percentage (%)'] = ATRIAL_FIB_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_PREINDEX)

print('=======================================')
ATRIAL_FLUTTER_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['ATRIAL_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_PREINDEX['percentage (%)'] = ATRIAL_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_PREINDEX)

print('=======================================')
AFIB_FLUTTER_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['AFIB_FLUTTER_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_PREINDEX['percentage (%)'] = AFIB_FLUTTER_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_PREINDEX)

print('=======================================')
HYPERTENSION_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['HYPERTENSION_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_PREINDEX['percentage (%)'] = HYPERTENSION_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_PREINDEX)

print('=======================================')
MI_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['MI_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_PREINDEX['percentage (%)'] = MI_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_PREINDEX)

print('=======================================')
DIABETES_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['DIABETES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_PREINDEX['percentage (%)'] = DIABETES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_PREINDEX)

print('=======================================')
DYSLIPIDEMIA_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['DYSLIPIDEMIA_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_PREINDEX['percentage (%)'] = DYSLIPIDEMIA_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_PREINDEX)

print('=======================================')
CHF_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['CHF_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_PREINDEX['percentage (%)'] = CHF_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_PREINDEX)

print('=======================================')
HYPERCOAG_STATES_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['HYPERCOAG_STATES_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_PREINDEX['percentage (%)'] = HYPERCOAG_STATES_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_PREINDEX)

print('=======================================')
MILD_LIVER_DIS_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['MILD_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_PREINDEX['percentage (%)'] = MILD_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_PREINDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['MOD_SEV_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_PREINDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LIVER_DIS_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['CHRONIC_LIVER_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_PREINDEX['percentage (%)'] = CHRONIC_LIVER_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_PREINDEX)

print('=======================================')
CHRONIC_LUNG_DIS_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['CHRONIC_LUNG_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_PREINDEX['percentage (%)'] = CHRONIC_LUNG_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_PREINDEX)

print('=======================================')
RHEUM_DIS_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['RHEUM_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_PREINDEX['percentage (%)'] = RHEUM_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_PREINDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['CHRONIC_KIDNEY_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_PREINDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_PREINDEX)

print('=======================================')
NEOPLASM_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['NEOPLASM_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_PREINDEX['percentage (%)'] = NEOPLASM_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_PREINDEX)

print('=======================================')
PERI_VASC_DIS_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['PERI_VASC_DIS_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_PREINDEX['percentage (%)'] = PERI_VASC_DIS_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_PREINDEX)

print('=======================================')
PFO_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['PFO_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_PREINDEX['percentage (%)'] = PFO_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_PREINDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['PAST_ISCHEMIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_PREINDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_PREINDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_PREINDEX =comorbidity_data_non_afib_5_year_followup.groupby(['PAST_HEMORRHAGIC_STROKE_PREINDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_PREINDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_PREINDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_PREINDEX)

   ATRIAL_FIB_PREINDEX  Patients  percentage (%)
0                    0      1302          100.00
   ATRIAL_FLUTTER_PREINDEX  Patients  percentage (%)
0                        0      1302          100.00
   AFIB_FLUTTER_PREINDEX  Patients  percentage (%)
0                      0      1302          100.00
   HYPERTENSION_PREINDEX  Patients  percentage (%)
0                      0       768           58.99
1                      1       534           41.01
   MI_PREINDEX  Patients  percentage (%)
0            0      1234           94.78
1            1        68            5.22
   DIABETES_PREINDEX  Patients  percentage (%)
0                  0      1066           81.87
1                  1       236           18.13
   DYSLIPIDEMIA_PREINDEX  Patients  percentage (%)
0                      0       825           63.36
1                      1       477           36.64
   CHF_PREINDEX  Patients  percentage (%)
0             0      1263           97.00
1             1        39            3.0

### CoMorbidity AT INDEX

In [43]:
ATRIAL_FIB_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['ATRIAL_FIB_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FIB_AT_INDEX['percentage (%)'] = ATRIAL_FIB_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FIB_AT_INDEX)

print('=======================================')
ATRIAL_FLUTTER_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['ATRIAL_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
ATRIAL_FLUTTER_AT_INDEX['percentage (%)'] = ATRIAL_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(ATRIAL_FLUTTER_AT_INDEX)

print('=======================================')
AFIB_FLUTTER_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['AFIB_FLUTTER_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
AFIB_FLUTTER_AT_INDEX['percentage (%)'] = AFIB_FLUTTER_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(AFIB_FLUTTER_AT_INDEX)

print('=======================================')
HYPERTENSION_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['HYPERTENSION_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERTENSION_AT_INDEX['percentage (%)'] = HYPERTENSION_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERTENSION_AT_INDEX)

print('=======================================')
MI_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['MI_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MI_AT_INDEX['percentage (%)'] = MI_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MI_AT_INDEX)

print('=======================================')
DIABETES_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['DIABETES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DIABETES_AT_INDEX['percentage (%)'] = DIABETES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DIABETES_AT_INDEX)

print('=======================================')
DYSLIPIDEMIA_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['DYSLIPIDEMIA_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
DYSLIPIDEMIA_AT_INDEX['percentage (%)'] = DYSLIPIDEMIA_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(DYSLIPIDEMIA_AT_INDEX)

print('=======================================')
CHF_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['CHF_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHF_AT_INDEX['percentage (%)'] = CHF_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHF_AT_INDEX)

print('=======================================')
HYPERCOAG_STATES_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['HYPERCOAG_STATES_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
HYPERCOAG_STATES_AT_INDEX['percentage (%)'] = HYPERCOAG_STATES_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(HYPERCOAG_STATES_AT_INDEX)

print('=======================================')
MILD_LIVER_DIS_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['MILD_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MILD_LIVER_DIS_AT_INDEX['percentage (%)'] = MILD_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MILD_LIVER_DIS_AT_INDEX)

print('=======================================')
MOD_SEV_LIVER_DIS_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['MOD_SEV_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
MOD_SEV_LIVER_DIS_AT_INDEX['percentage (%)'] = MOD_SEV_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(MOD_SEV_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LIVER_DIS_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['CHRONIC_LIVER_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LIVER_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LIVER_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LIVER_DIS_AT_INDEX)

print('=======================================')
CHRONIC_LUNG_DIS_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['CHRONIC_LUNG_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_LUNG_DIS_AT_INDEX['percentage (%)'] = CHRONIC_LUNG_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_LUNG_DIS_AT_INDEX)

print('=======================================')
RHEUM_DIS_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['RHEUM_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
RHEUM_DIS_AT_INDEX['percentage (%)'] = RHEUM_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(RHEUM_DIS_AT_INDEX)

print('=======================================')
CHRONIC_KIDNEY_DIS_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['CHRONIC_KIDNEY_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
CHRONIC_KIDNEY_DIS_AT_INDEX['percentage (%)'] = CHRONIC_KIDNEY_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(CHRONIC_KIDNEY_DIS_AT_INDEX)

print('=======================================')
NEOPLASM_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['NEOPLASM_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
NEOPLASM_AT_INDEX['percentage (%)'] = NEOPLASM_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(NEOPLASM_AT_INDEX)

print('=======================================')
PERI_VASC_DIS_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['PERI_VASC_DIS_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PERI_VASC_DIS_AT_INDEX['percentage (%)'] = PERI_VASC_DIS_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PERI_VASC_DIS_AT_INDEX)

print('=======================================')
PFO_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['PFO_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PFO_AT_INDEX['percentage (%)'] = PFO_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PFO_AT_INDEX)

print('=======================================')
PAST_ISCHEMIC_STROKE_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['PAST_ISCHEMIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_ISCHEMIC_STROKE_AT_INDEX['percentage (%)'] = PAST_ISCHEMIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_ISCHEMIC_STROKE_AT_INDEX)

print('=======================================')
PAST_HEMORRHAGIC_STROKE_AT_INDEX =comorbidity_data_non_afib_5_year_followup.groupby(['PAST_HEMORRHAGIC_STROKE_AT_INDEX']) \
       .agg({'PT_ID':'nunique'}) \
       .rename(columns={'PT_ID':'Patients'}) \
       .reset_index()
PAST_HEMORRHAGIC_STROKE_AT_INDEX['percentage (%)'] = PAST_HEMORRHAGIC_STROKE_AT_INDEX[['Patients']].apply(lambda x: x *100/ x.sum())
print(PAST_HEMORRHAGIC_STROKE_AT_INDEX)

   ATRIAL_FIB_AT_INDEX  Patients  percentage (%)
0                    0      1302          100.00
   ATRIAL_FLUTTER_AT_INDEX  Patients  percentage (%)
0                        0      1302          100.00
   AFIB_FLUTTER_AT_INDEX  Patients  percentage (%)
0                      0      1302          100.00
   HYPERTENSION_AT_INDEX  Patients  percentage (%)
0                      0       438           33.64
1                      1       864           66.36
   MI_AT_INDEX  Patients  percentage (%)
0            0      1196           91.86
1            1       106            8.14
   DIABETES_AT_INDEX  Patients  percentage (%)
0                  0       955           73.35
1                  1       347           26.65
   DYSLIPIDEMIA_AT_INDEX  Patients  percentage (%)
0                      0       532           40.86
1                      1       770           59.14
   CHF_AT_INDEX  Patients  percentage (%)
0             0      1249           95.93
1             1        53            4.0

In [44]:
data_full_non_afib_encounter_after_stroke[(data_full_non_afib_encounter_after_stroke['Difference_in_years'] > 3)&(data_full_non_afib_encounter_after_stroke['Difference_in_years'] > 5)].nunique()

PT_ID                  1302
ENC_DT                 3567
ENC_TYPE                  4
ICD_CD                 7641
ICD_NM                 7243
INDEX_DT               1069
Difference             3405
Difference_in_years    3405
dtype: int64